# Segmenting and clustering of East Toronto borough

In [111]:
import pandas as pd
import folium
import requests
from geopy.geocoders import Nominatim
import numpy as np

In [2]:
latlon=pd.read_csv('Toronto_lat_long.csv')
latlon.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,Regent Park / Harbourfront,43.654260,-79.360636
3,M6A,North York,Lawrence Manor / Lawrence Heights,43.718518,-79.464763
4,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government,43.662301,-79.389494


In [36]:
address = 'Toronto'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


In [24]:
toronto_map = folium.Map(location=[latitude, longitude], zoom_start=10) # generate map centred around Toronto


# plotting all the latitudes and longitudes on the Toronto map
for lat, lng, label in zip(latlon['Latitude'], latlon['Longitude'], latlon['Neighborhood']):
    folium.CircleMarker(
            [lat, lng],
            radius=5,
            fill=True,
            tooltip=label,
            color='blue',
            fill_color='blue',
            fill_opacity=0.6
        ).add_to(toronto_map)

In [25]:
toronto_map

In [14]:
latlon['Borough'].unique()

array(['North York', 'Downtown Toronto', 'Etobicoke', 'Scarborough',
       'East York', 'York', 'East Toronto', 'West Toronto',
       'Central Toronto', 'Mississauga'], dtype=object)

In [21]:
east=latlon[latlon['Borough']=='East Toronto'].reset_index(drop=True)
east

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M4E,East Toronto,The Beaches,43.676357,-79.293031
1,M4K,East Toronto,The Danforth West / Riverdale,43.679557,-79.352188
2,M4L,East Toronto,India Bazaar / The Beaches West,43.668999,-79.315572
3,M4M,East Toronto,Studio District,43.659526,-79.340923
4,M7Y,East Toronto,Business reply mail Processing CentrE,43.662744,-79.321558


In [34]:
toronto1_map = folium.Map(location=[latitude, longitude], zoom_start=13) # generate map centred around Toronto

# plotting all the latitudes and longitudes on the Toronto map
for lat, lng, label in zip(east['Latitude'], east['Longitude'], east['Neighborhood']):
    folium.CircleMarker(
            [lat, lng],
            radius=5,
            fill=True,
            tooltip=label,
            color='blue',
            fill_color='blue',
            fill_opacity=0.6
        ).add_to(toronto1_map)

toronto1_map

In [38]:
address = 'Studio district,Toronto'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
neighborhood_latitude = location.latitude
neighborhood_longitude = location.longitude
print('The geograpical coordinate of {}, are {}, {}.'.format(address,neighborhood_latitude, neighborhood_longitude))

The geograpical coordinate of Studio district,Toronto, are 43.64958515, -79.39068322559326.


In [39]:
CLIENT_ID = 'RGZJWXE322CDIC0RGBW2ZOQ4KKAJZMI2Y00TUZEVBRHLY1ZF' # your Foursquare ID
CLIENT_SECRET = 'IDAK1MBJQWDKKEFRFNNYN0T0GN0EWXZI4NKG1HBIJCJVRI4H' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: RGZJWXE322CDIC0RGBW2ZOQ4KKAJZMI2Y00TUZEVBRHLY1ZF
CLIENT_SECRET:IDAK1MBJQWDKKEFRFNNYN0T0GN0EWXZI4NKG1HBIJCJVRI4H


In [40]:
LIMIT=100

radius=500

url='https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?&client_id=RGZJWXE322CDIC0RGBW2ZOQ4KKAJZMI2Y00TUZEVBRHLY1ZF&client_secret=IDAK1MBJQWDKKEFRFNNYN0T0GN0EWXZI4NKG1HBIJCJVRI4H&v=20180605&ll=43.64958515,-79.39068322559326&radius=500&limit=100'

In [41]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5e959a73963d29001b9b6d7a'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Entertainment District',
  'headerFullLocation': 'Entertainment District, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 139,
  'suggestedBounds': {'ne': {'lat': 43.6540851545, 'lng': -79.3844757089414},
   'sw': {'lat': 43.6450851455, 'lng': -79.39689074224513}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4b7ed424f964a5208a0230e3',
       'name': 'Friendly Stranger - Cannabis Culture Shop',
       'location': {'address': '241 Queen St. W',
        'crossStreet': 'at St. Patrick St.',
        'lat': 43.650386666114635,
        'lng': -79.

In [42]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [44]:
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

In [45]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Friendly Stranger - Cannabis Culture Shop,Smoke Shop,43.650387,-79.388523
1,DAVIDsTEA,Tea Room,43.649698,-79.392444
2,YYoga,Yoga Studio,43.649725,-79.391983
3,JaBistro,Sushi Restaurant,43.649687,-79.388090
4,Dark Horse Espresso Bar,Coffee Shop,43.648292,-79.390390


In [46]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

100 venues were returned by Foursquare.


In [47]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [49]:
east_toronto_venues = getNearbyVenues(names=east['Neighborhood'],
                                   latitudes=east['Latitude'],
                                   longitudes=east['Longitude']
                                  )


The Beaches
The Danforth West / Riverdale
India Bazaar / The Beaches West
Studio District
Business reply mail Processing CentrE


In [50]:
print(east_toronto_venues.shape)
east_toronto_venues.head()

(123, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,The Beaches,43.676357,-79.293031,Glen Manor Ravine,43.676821,-79.293942,Trail
1,The Beaches,43.676357,-79.293031,The Big Carrot Natural Food Market,43.678879,-79.297734,Health Food Store
2,The Beaches,43.676357,-79.293031,Grover Pub and Grub,43.679181,-79.297215,Pub
3,The Beaches,43.676357,-79.293031,Upper Beaches,43.680563,-79.292869,Neighborhood
4,The Danforth West / Riverdale,43.679557,-79.352188,Pantheon,43.677621,-79.351434,Greek Restaurant


In [98]:
east_toronto_venues['Venue Category'].unique()

array(['Trail', 'Health Food Store', 'Pub', 'Neighborhood',
       'Greek Restaurant', 'Cosmetics Shop', 'Italian Restaurant',
       'Ice Cream Shop', 'Yoga Studio', 'Brewery',
       'Fruit & Vegetable Store', 'Pizza Place', 'Bookstore',
       'Restaurant', 'Dessert Shop', 'Juice Bar', 'Bubble Tea Shop',
       'Spa', 'Diner', 'Grocery Store', 'Furniture / Home Store', 'Café',
       'Coffee Shop', 'Bakery', 'Caribbean Restaurant',
       'Indian Restaurant', 'American Restaurant', 'Frozen Yogurt Shop',
       'Lounge', 'Liquor Store', 'Gym', 'Fish & Chips Shop',
       'Fast Food Restaurant', 'Sushi Restaurant', 'Park', 'Pet Store',
       'Steakhouse', 'Burrito Place', 'Movie Theater', 'Sandwich Place',
       'Fish Market', 'Gay Bar', 'Cheese Shop',
       'Middle Eastern Restaurant', 'Comfort Food Restaurant',
       'Thai Restaurant', 'Seafood Restaurant', 'Stationery Store',
       'Coworking Space', 'Wine Bar', 'Bar', 'Gym / Fitness Center',
       'Latin American Restaurant'

In [54]:
east_toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Business reply mail Processing CentrE,16,16,16,16,16,16
India Bazaar / The Beaches West,20,20,20,20,20,20
Studio District,40,40,40,40,40,40
The Beaches,4,4,4,4,4,4
The Danforth West / Riverdale,43,43,43,43,43,43


In [99]:
#analysing each neighborhood
#there is already a type of venue category called neighborhood. So the neighborhoods column in east toronto is renamed 
#in the dataframe

# one hot encoding
e_toronto_onehot = pd.get_dummies(east_toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
e_toronto_onehot['Venue_Neighborhood'] = east_toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [e_toronto_onehot.columns[-1]] + list(e_toronto_onehot.columns[:-1])
e_toronto_onehot = e_toronto_onehot[fixed_columns]

e_toronto_onehot.head()

,Venue_Neighborhood,American Restaurant,Auto Workshop,Bakery,Bank,Bar,Bookstore,Brewery,Bubble Tea Shop,Burrito Place,...,Skate Park,Smoke Shop,Spa,Stationery Store,Steakhouse,Sushi Restaurant,Thai Restaurant,Trail,Wine Bar,Yoga Studio
0,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
1,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,The Danforth West / Riverdale,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [102]:
e_toronto_onehot.shape

(123, 67)

In [105]:
grouped_east_toronto = e_toronto_onehot.groupby('Venue_Neighborhood').mean().reset_index()
grouped_east_toronto

,Venue_Neighborhood,American Restaurant,Auto Workshop,Bakery,Bank,Bar,Bookstore,Brewery,Bubble Tea Shop,Burrito Place,...,Skate Park,Smoke Shop,Spa,Stationery Store,Steakhouse,Sushi Restaurant,Thai Restaurant,Trail,Wine Bar,Yoga Studio
0,Business reply mail Processing CentrE,0.000000,0.0625,0.000000,0.000,0.000,0.000000,0.062500,0.000000,0.0625,...,0.0625,0.0625,0.062500,0.000,0.00,0.00,0.000,0.000000,0.000,0.000000
1,India Bazaar / The Beaches West,0.000000,0.0000,0.000000,0.000,0.000,0.000000,0.050000,0.000000,0.0500,...,0.0000,0.0000,0.000000,0.000,0.05,0.05,0.000,0.000000,0.000,0.000000
2,Studio District,0.050000,0.0000,0.050000,0.025,0.025,0.025000,0.050000,0.000000,0.0000,...,0.0000,0.0000,0.000000,0.025,0.00,0.00,0.025,0.000000,0.025,0.025000
3,The Beaches,0.000000,0.0000,0.000000,0.000,0.000,0.000000,0.000000,0.000000,0.0000,...,0.0000,0.0000,0.000000,0.000,0.00,0.00,0.000,0.250000,0.000,0.000000
4,The Danforth West / Riverdale,0.023256,0.0000,0.023256,0.000,0.000,0.046512,0.023256,0.023256,0.0000,...,0.0000,0.0000,0.023256,0.000,0.00,0.00,0.000,0.023256,0.000,0.023256


In [106]:
grouped_east_toronto.shape

(5, 67)

In [108]:
#top 5 most common venues in each neighborhood
num_top_venues = 5

for hood in grouped_east_toronto['Venue_Neighborhood']:
    print("----"+hood+"----")
    temp = grouped_east_toronto[grouped_east_toronto['Venue_Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Business reply mail Processing CentrE----
            venue  freq
0      Smoke Shop  0.06
1   Garden Center  0.06
2  Farmers Market  0.06
3            Park  0.06
4     Pizza Place  0.06


----India Bazaar / The Beaches West----
                  venue  freq
0                  Park  0.15
1  Fast Food Restaurant  0.10
2           Pizza Place  0.05
3         Movie Theater  0.05
4             Pet Store  0.05


----Studio District----
                 venue  freq
0                 Café  0.10
1          Coffee Shop  0.08
2              Brewery  0.05
3  American Restaurant  0.05
4            Gastropub  0.05


----The Beaches----
                 venue  freq
0                Trail  0.25
1                  Pub  0.25
2    Health Food Store  0.25
3         Neighborhood  0.25
4  American Restaurant  0.00


----The Danforth West / Riverdale----
                venue  freq
0    Greek Restaurant  0.19
1  Italian Restaurant  0.07
2         Coffee Shop  0.07
3          Restaurant  0.05
4           

In [109]:
#sort the venues in descending order

def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [113]:
#lets display top 10 venues in each neighborhood

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = grouped_east_toronto['Venue_Neighborhood']

for ind in np.arange(grouped_east_toronto.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(grouped_east_toronto.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Business reply mail Processing CentrE,Garden,Brewery,Light Rail Station,Farmers Market,Park,Butcher,Pizza Place,Burrito Place,Comic Shop,Fast Food Restaurant
1,India Bazaar / The Beaches West,Park,Fast Food Restaurant,Fish & Chips Shop,Restaurant,Ice Cream Shop,Liquor Store,Gym,Movie Theater,Burrito Place,Pet Store
2,Studio District,Café,Coffee Shop,Brewery,Gastropub,American Restaurant,Bakery,Bookstore,Bar,Bank,Cheese Shop
3,The Beaches,Neighborhood,Trail,Pub,Health Food Store,Farmers Market,Convenience Store,Cosmetics Shop,Coworking Space,Dessert Shop,Diner
4,The Danforth West / Riverdale,Greek Restaurant,Italian Restaurant,Coffee Shop,Restaurant,Bookstore,Ice Cream Shop,Furniture / Home Store,Indian Restaurant,Grocery Store,Fruit & Vegetable Store


## Clustering using K-means

In [115]:
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

In [117]:
# set number of clusters
kclusters = 3

grouped_east_toronto_clustering = grouped_east_toronto.drop('Venue_Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(grouped_east_toronto_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([2, 2, 0, 1, 0])

In [119]:
#lets create a new df that has clusters column as well

# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

grouped_east_toronto = east

# merge grouped_east_toronto with east data to add latitude/longitude for each neighborhood
grouped_east_toronto = grouped_east_toronto.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')



ValueError: cannot insert Cluster Labels, already exists

In [120]:
grouped_east_toronto.head() # check the last columns!

,Postal Code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4E,East Toronto,The Beaches,43.676357,-79.293031,1,Neighborhood,Trail,Pub,Health Food Store,Farmers Market,Convenience Store,Cosmetics Shop,Coworking Space,Dessert Shop,Diner
1,M4K,East Toronto,The Danforth West / Riverdale,43.679557,-79.352188,0,Greek Restaurant,Italian Restaurant,Coffee Shop,Restaurant,Bookstore,Ice Cream Shop,Furniture / Home Store,Indian Restaurant,Grocery Store,Fruit & Vegetable Store
2,M4L,East Toronto,India Bazaar / The Beaches West,43.668999,-79.315572,2,Park,Fast Food Restaurant,Fish & Chips Shop,Restaurant,Ice Cream Shop,Liquor Store,Gym,Movie Theater,Burrito Place,Pet Store
3,M4M,East Toronto,Studio District,43.659526,-79.340923,0,Café,Coffee Shop,Brewery,Gastropub,American Restaurant,Bakery,Bookstore,Bar,Bank,Cheese Shop
4,M7Y,East Toronto,Business reply mail Processing CentrE,43.662744,-79.321558,2,Garden,Brewery,Light Rail Station,Farmers Market,Park,Butcher,Pizza Place,Burrito Place,Comic Shop,Fast Food Restaurant


In [122]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(grouped_east_toronto['Latitude'], grouped_east_toronto['Longitude'], grouped_east_toronto['Neighborhood'], grouped_east_toronto['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [128]:
grouped_east_toronto.loc[grouped_east_toronto['Cluster Labels'] == 0, grouped_east_toronto.columns[[1] +[2]+ list(range(5, grouped_east_toronto.shape[1]))]]

,Borough,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,East Toronto,The Danforth West / Riverdale,0,Greek Restaurant,Italian Restaurant,Coffee Shop,Restaurant,Bookstore,Ice Cream Shop,Furniture / Home Store,Indian Restaurant,Grocery Store,Fruit & Vegetable Store
3,East Toronto,Studio District,0,Café,Coffee Shop,Brewery,Gastropub,American Restaurant,Bakery,Bookstore,Bar,Bank,Cheese Shop


In [129]:
grouped_east_toronto.loc[grouped_east_toronto['Cluster Labels'] == 1, grouped_east_toronto.columns[[1] +[2]+ list(range(5, grouped_east_toronto.shape[1]))]]

,Borough,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,East Toronto,The Beaches,1,Neighborhood,Trail,Pub,Health Food Store,Farmers Market,Convenience Store,Cosmetics Shop,Coworking Space,Dessert Shop,Diner


In [130]:
grouped_east_toronto.loc[grouped_east_toronto['Cluster Labels'] == 2, grouped_east_toronto.columns[[1] +[2]+ list(range(5, grouped_east_toronto.shape[1]))]]

,Borough,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,East Toronto,India Bazaar / The Beaches West,2,Park,Fast Food Restaurant,Fish & Chips Shop,Restaurant,Ice Cream Shop,Liquor Store,Gym,Movie Theater,Burrito Place,Pet Store
4,East Toronto,Business reply mail Processing CentrE,2,Garden,Brewery,Light Rail Station,Farmers Market,Park,Butcher,Pizza Place,Burrito Place,Comic Shop,Fast Food Restaurant
